# Implementation - Graphical model

[```igraph```](https://github.com/igraph/python-igraph) package is required since it is used for internal representation of a probabilistic graphical model. [```pyvis```](https://pyvis.readthedocs.io) is optional and it is needed for creating fancy interactive visualizations.

In [1]:
import numpy  as np
import igraph as ig
%run ./2\ Implementation\ -\ Factors\ and\ operations.ipynb
# optional package
import pyvis.network as net

## 1 PGM

We are going to stick with factor graphs since every Bayesian network and Markov network can be converted to this representation.

### 1.1 Factor graph data structure

As already mentioned the core of factor graph data structure is an ```igraph``` graph. Each node necessarily has the following attributes:

* ```name``` attribute serves as a unique ```string``` identifier either of a factor node or a variable node
* ```is_factor``` is a boolean indicator
* ```factor_``` attribute is `None` for a variable node and stores a ```factor``` data structure for a factor node

In [2]:
class factor_graph:
    def __init__(self):
        self._graph = ig.Graph()
    
    # ----------------------- Factor node actions -----------
    def add_factor_node(self, f_name, factor_):
        if self.__check_node(f_name, exception=True) != False:
            raise Exception('Invalid name')
        if f_name in factor_.get_variables():
            raise Exception('Invalid factor name')
        if type(factor_) is not factor:
            raise Exception('Data is not factor')
            
        for v_name in factor_.get_variables():
            if self.__check_node(v_name) == False:
                self.__create_variable_node(v_name)
            
        self._graph.add_vertex(f_name)
        self._graph.vs.find(name=f_name)['is_factor'] = True
        self._graph.vs.find(name=f_name)['factor_']   = factor_
            
        start = self._graph.vs.find(name=f_name).index
        edge_list = [tuple([start, self._graph.vs.find(name=i).index]) for i in factor_.get_variables()]
        self._graph.add_edges(edge_list)
        
    #def change_factor(self, name, factor_, remove_zero_deg_var=False):
        
    def remove_factor(self, f_name, remove_zero_degree=False):
        self.__check_node(f_name, exception=True, target='factor')
        
        f_neighbors = self._graph.neighbors(f_name, mode="out")
        g.delete_vertices(f_name)
        
        if remove_zero_degree:
            for v_name in factor_neighbors:
                if g.vs.find(v_name).degree() == 0:
                    remove_variable(self, v_name)
        
    
    # ----------------------- Variable node actions ---------
    def add_variable_node(self, v_name):
        self.__check_node(v_name, exception=True)
        self.__create_variable_node(v_name)
    
    def remove_variable(self, v_name):
        self.__check_node(v_name, exception=True, target='variable')
        if self._graph.vs.find(v_name).degree() == 0:
            self._graph.delete_vertices(self._graph.vs.find(v_name).index)
        else:
            raise Exception('Can not delete variables with degree >0')
            
    def __create_variable_node(self, v_name):
        self._graph.add_vertex(v_name)
        self._graph.vs.find(name=v_name)['is_factor'] = False
        
    # ----------------------- Info --------------------------
    def get_node_status(self, name):
        return self.__check_node(name)
    
    def __check_node(self, name, exception=False, target=None):
        if len(self._graph.vs) == 0:
            return False
        elif len(self._graph.vs.select(name_eq=name)) == 0:
            return False
        else:
            if self._graph.vs.find(name=name)['is_factor'] == True:
                if (target == 'variable') and exception: raise Exception('Factor name. Variable expected')
                return 'factor'
            else:
                if (target == 'factor') and exception: raise Exception('Variable name. Factor expected')
                return 'variable'
    
    # ----------------------- Graph structure ---------------
    def get_graph(self):
        return self._graph
    
    def is_connected(self):
        return self._graph.is_connected()
        
    def is_tree(self):
        return self._graph.is_tree()

#### Example

Let's create a simple graphical model.

In [3]:
pgm_1 = factor_graph()
pgm_1.add_factor_node('p1', factor(['x1', 'x2', 'x3']))
pgm_1.add_factor_node('p2', factor(['x2', 'x4']))

### 1.2 Factor graph from string

In [4]:
def string2factor_graph(str_):
    res_factor_graph = factor_graph()
    
    str_ = [i.split('(') for i in str_.split(')') if i != '']
    for i in range(len(str_)):
        str_[i][1] = str_[i][1].split(',')
        
    for i in str_:
        res_factor_graph.add_factor_node(i[0], factor(i[1]))
    
    return res_factor_graph

#### Example

In [5]:
pgm_2 = string2factor_graph('phi_1(a,b,c)phi_2(b,c,d,e)psi_3(e,c)psi_4(d)')

---

## 2 Plotting

The default way of plotting internal graph is ```plot(x.get_graph())```. ```plot_factor_graph``` does interactive plotting using the ```pyvis``` package.

In [6]:
def plot_factor_graph(x):
    graph = net.Network(notebook=True, width="100%")
    graph.toggle_physics(False)
    
    # Vertices
    label = x.get_graph().vs['name']
    color = ['#2E2E2E' if i is True else '#F2F2F2' for i in x.get_graph().vs['is_factor']]
    graph.add_nodes(range(len(x.get_graph().vs)), label=label, color=color)
    
    # Edges
    graph.add_edges(x.get_graph().get_edgelist())
    
    return graph.show("./img/3 Implementation - Graphical model/graph.html")

#### Example

In [7]:
plot_factor_graph(pgm_2)